In [1]:
import pandas as pd
import statistics
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.dummy import DummyRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score

In [2]:
####### get feature table for ml
###### half-life fold change values in hypoxia to log phase for leadered transcript

hl_c4_raw = pd.read_csv("../../feature/FeatureTables/featureTable_combinedSelected_halfLifeValues/HalfLifeFc_hypoxiaToLogPhase_combinedSelected_leadered_byLeaderedLogPhase_jointlyByLogPhaseFoldChange.csv", header = 0)

hl_c4 = hl_c4_raw.iloc[:, 2:]
hl_c4 = hl_c4.dropna()
X_c4 = hl_c4.iloc[:, :-1]
y_c4 = hl_c4.HalfLife_FC_hypoxiaToLogPhase

print(X_c4.shape)
print(X_c4.shape[1])
print(y_c4.shape)
# X_c4.head()
# y_c4.value_counts()

(1026, 277)
277
(1026,)


In [3]:
####### K-Folds split

def kf(X, y, n_fold):
    train_kf_idex = []
    test_kf_idex = []
    kf = KFold(n_splits = n_fold, shuffle = True)

    for train_index, test_index in kf.split(X, y):
        train_kf_idex.append(train_index)
        test_kf_idex.append(test_index)
    return train_kf_idex, test_kf_idex

In [4]:
####### k fold cross-validation

def cv_training(X, y, n_fold):
    
    train_idex, test_idex = kf(X, y, n_fold)
    
    r2 = {}
    for regr in ['MeanRegression', 'LinearRegression', 'LassoRegression', 'SupportVector', 'DecisionTree',
                'RandomForest']:
        r2[regr] = []
    
    for i in range(n_fold):
        X_train = X.iloc[train_idex[i]]
        y_train = y.iloc[train_idex[i]]
        X_test = X.iloc[test_idex[i]]
        y_test = y.iloc[test_idex[i]]
        
        std_scaler = preprocessing.StandardScaler()
        X_train_std = std_scaler.fit_transform(X_train)
        X_test_std = std_scaler.transform(X_test)
        
        regr_mean = DummyRegressor(strategy = 'mean')
        regr_mean.fit(X_train, y_train)
        y_pred_strat = regr_mean.predict(X_test)
        r2_mean = r2_score(y_test, y_pred_strat, multioutput = 'uniform_average')
        r2['MeanRegression'].append(r2_mean)
        
        regr_liner = LinearRegression()
        regr_liner.fit(X_train_std, y_train)
        y_pred_liner = regr_liner.predict(X_test_std)
        r2_liner = r2_score(y_test, y_pred_liner, multioutput = 'uniform_average')
        r2['LinearRegression'].append(r2_liner)
        
        regr_lasso = linear_model.Lasso(alpha=0.1)
        regr_lasso.fit(X_train_std, y_train)
        y_pred_lasso = regr_lasso.predict(X_test_std)
        r2_lasso = r2_score(y_test, y_pred_liner, multioutput = 'uniform_average')
        r2['LassoRegression'].append(r2_lasso)

        regr_svm = svm.SVR(kernel = 'rbf')
        regr_svm.fit(X_train_std, y_train)
        y_pred_svm = regr_svm.predict(X_test_std)
        r2_svm = r2_score(y_test, y_pred_svm, multioutput = 'uniform_average')
        r2['SupportVector'].append(r2_svm)
        
        regr_dt = DecisionTreeRegressor()
        regr_dt.fit(X_train, y_train)
        y_pred_dt = regr_dt.predict(X_test)
        r2_dt = r2_score(y_test, y_pred_dt, multioutput = 'uniform_average')
        r2['DecisionTree'].append(r2_dt)
        
        regr_rf = RandomForestRegressor(n_jobs = -1)
        regr_rf.fit(X_train, y_train)
        y_pred_rf = regr_rf.predict(X_test)
        r2_rf = r2_score(y_test, y_pred_rf, multioutput = 'uniform_average')
        r2['RandomForest'].append(r2_rf)
        
    return r2

In [5]:
####### repeated runs

def repeat_cv_training(X, y, n_fold, n_times):
    
    r2_repeat = {}
    for regr in ['MeanRegression', 'LinearRegression', 'LassoRegression', 'SupportVector', 'DecisionTree',
                'RandomForest']:
        r2_repeat[regr] = []
        
    for i in range(n_times):
        print('training: ' + str(i))
        cv_r2_i = cv_training(X, y, n_fold)
        for regr, r2 in cv_r2_i.items():
            r2_repeat[regr].append(r2)
            
    return r2_repeat

In [6]:
####### test with single run

cv_r2 = cv_training(X_c4, y_c4, 5)
for regr, r2 in cv_r2.items():
    print("%s\t%.4f\t%.4f" % (regr, statistics.mean(r2), statistics.stdev(r2)))

/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.371e+05, tolerance: 1.321e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.593e+05, tolerance: 1.355e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

MeanRegression	-0.1076	0.2281
LinearRegression	-11.9728	20.4216
LassoRegression	-11.9728	20.4216
SupportVector	-0.0217	0.0129
DecisionTree	-29.2534	50.7631
RandomForest	-6.5295	10.9975


In [7]:
####### running 5 fold cross-validation, repeated 10 times

cv_r2_repeat = repeat_cv_training(X_c4, y_c4, 5, 10)
# print(cv_r2_repeat)

training: 0


/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.811e+05, tolerance: 1.352e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.778e+03, tolerance: 2.261e+03
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

training: 1


/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.038e+05, tolerance: 1.335e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.530e+05, tolerance: 1.311e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

training: 2


...

/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.346e+06, tolerance: 1.308e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.893e+05, tolerance: 1.355e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

training: 8


/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.691e+05, tolerance: 1.322e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.761e+05, tolerance: 1.348e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

training: 9


/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.738e+05, tolerance: 1.306e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.614e+05, tolerance: 1.348e+04
  model = cd_fast.enet_coordinate_descent(
/Users/huamingsun/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

In [8]:
####### get averaged R2 for each regressor

for regr, r2 in cv_r2_repeat.items():
    r2_flat = [r2_ij for r2_i in r2 for r2_ij in r2_i]
    print(regr, statistics.mean(r2_flat), min(r2_flat), max(r2_flat), statistics.stdev(r2_flat))

MeanRegression -0.10618943458174285 -0.930988024739396 -1.4976279587575192e-06 0.20678326051711074
LinearRegression -1.1791217445288236e+20 -5.895608722644118e+21 -0.008151350567712212 8.33764981400843e+20
LassoRegression -1.1791217445288236e+20 -5.895608722644118e+21 -0.008151350567712212 8.33764981400843e+20
SupportVector -0.026573512689701574 -0.06643932875212522 -0.006855501033893985 0.015118409600290206
DecisionTree -79.85150131941242 -1151.8758155663575 -0.013626984970399603 194.134159490229
RandomForest -6.312277138027466 -86.76977417232563 -0.00880012948023201 13.360607049586498
